In [1]:
from threading import Lock
status = 0
status_lock = Lock()

will_end = False
will_end_lock = Lock()

offset=[0,0]
offset_lock = Lock()


In [2]:
import keyboard
from threading import Thread
def monitor_keyboard():
    def callback_p():
        global status
        status_lock.acquire()
        status = 1 - status
        status_lock.release()
        # print("aimbot status: {}".format("on" if status else "off"))
    def callback_up():
        global offset
        offset_lock.acquire()
        offset[1]-=10
        offset_lock.release()
    def callback_down():
        global offset
        offset_lock.acquire()
        offset[1]+=10
        offset_lock.release()
    def callback_left():
        global offset
        offset_lock.acquire()
        offset[0]-=10
        offset_lock.release()
    def callback_right():
        global offset
        offset_lock.acquire()
        offset[0]+=10
        offset_lock.release()
    keyboard.add_hotkey('p',callback_p)
    keyboard.add_hotkey('up',callback_up)
    keyboard.add_hotkey('down',callback_down)
    keyboard.add_hotkey('left',callback_left)
    keyboard.add_hotkey('right',callback_right)
    keyboard.wait('o')
    global will_end
    will_end_lock.acquire()
    will_end = True
    will_end_lock.release()
    return
thread1 = Thread(target=monitor_keyboard)
thread1.start()
# thread1.join()

In [4]:
import pydirectinput
from math import sqrt
class Aim :
    global offset
    def __init__(self,cx,cy) -> None:
        self.cx = int(cx)
        self.cy = int(cy)
        self.id = -1

    def update(self,xywh,id):
        if id == None:
            return
        if self.id in id :
            for ind,(x,y,_,_) in enumerate(xywh):
                if id[ind] == self.id:
                    self.cx = x
                    self.cy = y
                    return
        candidate = None
        length = 10000000
        for ind,(x,y,_,_) in enumerate(xywh):
            if (x-320)**2+(y-240)**2<length:
                length = (x-320)**2+(y-240)**2
                candidate = ind
        self.cx,self.cy,_,_ = xywh[candidate]
        self.id = id[candidate]

    def move(self):
        x,y = pydirectinput.position()
        
        off_x = 320-self.cx
        off_y = 240-self.cy

        length = sqrt(off_x**2+off_y**2)
        w = 1600 / length
        w = max(w,20)
        w = min(w,6)

        off_x = int(off_x/w)
        off_y = int(off_y/w)
        pydirectinput.moveTo(x-off_x,y-off_y,relative=True)


In [5]:
from ultralytics import YOLO
import cv2
import numpy as np

cap = cv2.VideoCapture(1)
model = YOLO("yolov8n.pt")
aim = Aim(*pydirectinput.position())
while cap.isOpened():
    ret, frame = cap.read()
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    result = model.track(
        frame,
        show=False,
        persist=True,
        tracker="bytetrack.yaml",
        conf=0.3,
        classes=[
            0,
        ],
        verbose=False,
    )

    aim.update(result[0].boxes.xywh,result[0].boxes.id)
    cx = aim.cx
    cy = aim.cy
    id = aim.id

    annotated_frame: np.ndarray = result[0].plot(im_gpu=True, font_size=3, line_width=1)
    cv2.circle(annotated_frame, (320, 240), 1, (0, 255, 0), 5)
    cv2.circle(annotated_frame, (int(cx), int(cy)), 1, (255, 0, 0), 5)
    cv2.imshow("test", annotated_frame)
    key = cv2.waitKey(1)
    
    print(
        "\rframe height:{}\t width:{}\tfps:{}\tcx:{}\tcy:{}\tstatus:{}\toffset:{}\tid:{}\tpos:{}\tspeed:{}".format(
            height, width, fps, int(cx), int(cy), status, offset, id, pydirectinput.position(),list(map(lambda x:int(result[0].speed[x]),result[0].speed))
        ),
        end="",
        flush=True,
    )

    # status_lock.acquire()
    if status and result[0].boxes.id != None:
        aim.move()
        # aim.rel_move()
    # status_lock.release()
    
    # will_end_lock.acquire()
    if will_end:
        # will_end_lock.release()
        print("aimbot end")
        break
    # wioll_end_lock.release()


cap.release()
cv2.destroyAllWindows()
thread1.join()

frame height:480.0	 width:640.0	fps:0.0	cx:203	cy:172	status:0	offset:[0, 0]	id:613.0	pos:(1288, 666)	speed:[1, 11, 2]]]]aimbot end


here
